In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import faker as faker

Re-Engagement Score = (Engagement Score * 0.4) + (Increase in FX Hedging Needs * 20) + (New Plant Overseas * 15) + (New Currency Pair Needed * 15) + (100 - Economic Risk Score * 0.1)

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder

fake = Faker()

np.random.seed(42)

def generate_client_dataset(num_clients=1000):
    data = {
        'client_id': range(1, num_clients + 1),
        'company_name': [fake.company() for _ in range(num_clients)],
        'industry': np.random.choice([
            'Manufacturing', 'Technology', 'Finance',
            'Healthcare', 'Logistics', 'Energy'
        ], num_clients),
        'annual_revenue': np.random.uniform(1000000, 50000000, num_clients),
        'years_in_business': np.random.randint(1, 30, num_clients),

        'international_operations': np.random.choice([0, 1], num_clients, p=[0.4, 0.6]),
        'current_fx_hedging': np.random.choice([0, 1], num_clients, p=[0.3, 0.7]),

        'market_volatility_index': np.random.uniform(10, 50, num_clients),
        'currency_exposure_risk': np.random.uniform(0, 100, num_clients),

        'likelihood_to_hedge': np.random.choice([0, 1], num_clients, p=[0.4, 0.6])
    }

    df = pd.DataFrame(data)

    df['revenue_to_years_ratio'] = df['annual_revenue'] / df['years_in_business']
    df['risk_score'] = (df['market_volatility_index'] + df['currency_exposure_risk']) / 2

    return df

df = generate_client_dataset()

X = df.drop(['client_id', 'company_name', 'likelihood_to_hedge'], axis=1)
y = df['likelihood_to_hedge']

le = LabelEncoder()
X['industry'] = le.fit_transform(X['industry'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
knn_pred = knn.predict(X_test_scaled)
print("KNN Classification Report:")
print(classification_report(y_test, knn_pred))

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

nn_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
print("\nNeural Network Classification Report:")
print(classification_report(y_test, nn_pred))

potential_targets = df[df['likelihood_to_hedge'] == 1]
potential_targets.to_csv('potential_hedging_targets.csv', index=False)

KNN Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.32      0.32        75
           1       0.60      0.61      0.60       125

    accuracy                           0.50       200
   macro avg       0.46      0.46      0.46       200
weighted avg       0.50      0.50      0.50       200



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Neural Network Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.03      0.05        75
           1       0.62      0.97      0.76       125

    accuracy                           0.61       200
   macro avg       0.48      0.50      0.40       200
weighted avg       0.51      0.61      0.49       200



In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def create_interactive_visualizations(df, X_train_scaled, y_train, model, scaler):
    fig1 = px.scatter(
        df,
        x='annual_revenue',
        y='currency_exposure_risk',
        color='likelihood_to_hedge',
        hover_data=['company_name', 'industry'],
        title='Client Hedging Likelihood by Revenue and Currency Risk',
        labels={
            'annual_revenue': 'Annual Revenue ($)',
            'currency_exposure_risk': 'Currency Exposure Risk',
            'likelihood_to_hedge': 'Likely to Hedge'
        }
    )
    fig1.write_html("revenue_risk_scatter.html")

    fig2 = go.Figure()
    fig2.add_trace(go.Histogram(
        x=df[df['likelihood_to_hedge'] == 1]['annual_revenue'],
        name='Likely to Hedge',
        opacity=0.7
    ))
    fig2.add_trace(go.Histogram(
        x=df[df['likelihood_to_hedge'] == 0]['annual_revenue'],
        name='Not Likely to Hedge',
        opacity=0.7
    ))
    fig2.update_layout(
        title='Revenue Distribution by Hedging Likelihood',
        xaxis_title='Annual Revenue',
        yaxis_title='Frequency',
        barmode='overlay'
    )
    fig2.write_html("revenue_distribution.html")

    feature_importance = {}
    for i, feature in enumerate(X.columns):
        weights = np.abs(model.layers[0].get_weights()[0][:, i])
        feature_importance[feature] = np.mean(weights)

    feature_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns=['Importance'])
    feature_df = feature_df.reset_index().rename(columns={'index': 'Feature'})

    fig3 = px.bar(
        feature_df.sort_values('Importance', ascending=False),
        x='Feature',
        y='Importance',
        title='Feature Importance in Hedging Prediction'
    )
    fig3.write_html("feature_importance.html")

    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_train_scaled)

    y_pred_proba = model.predict(X_train_scaled).flatten()

    fig4 = px.scatter(
        x=X_pca[:, 0],
        y=X_pca[:, 1],
        color=y_train,
        color_continuous_scale='Viridis',
        title='Client Clustering in Reduced Dimension',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'}
    )
    fig4.update_traces(marker=dict(size=10))
    fig4.write_html("decision_boundary.html")

    return fig1, fig2, fig3, fig4

def generate_client_insights(df):
    insights = {
        'total_clients': len(df),
        'potential_hedge_clients': len(df[df['likelihood_to_hedge'] == 1]),
        'potential_hedge_percentage': len(df[df['likelihood_to_hedge'] == 1]) / len(df) * 100,
        'avg_revenue_hedge_clients': df[df['likelihood_to_hedge'] == 1]['annual_revenue'].mean(),
        'top_industries_for_hedging': df[df['likelihood_to_hedge'] == 1]['industry'].value_counts().to_dict()
    }

    import json
    with open('client_hedging_insights.json', 'w') as f:
        json.dump(insights, f, indent=4)

    return insights

def create_dashboard(visualizations):
    dashboard = go.Figure()
    dashboard.write_html("hedging_dashboard.html")

try:
    visualizations = create_interactive_visualizations(df, X_train_scaled, y_train, model, scaler)

    insights = generate_client_insights(df)

    print("Client Hedging Insights:")
    for key, value in insights.items():
        print(f"{key}: {value}")

    create_dashboard(visualizations)

except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Client Hedging Insights:
total_clients: 1000
potential_hedge_clients: 618
potential_hedge_percentage: 61.8
avg_revenue_hedge_clients: 25247427.09054073
top_industries_for_hedging: {'Logistics': 114, 'Healthcare': 113, 'Manufacturing': 113, 'Technology': 95, 'Energy': 92, 'Finance': 91}


In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np

def create_comprehensive_insights_visualization(df):
    # Prepare data for visualization
    insights_df = df.copy()

    # Create additional calculated columns
    insights_df['risk_score'] = (insights_df['market_volatility_index'] +
                                  insights_df['currency_exposure_risk']) / 2

    # Create interactive scatter plot with multiple dimensions
    fig = px.scatter(
        insights_df,
        x='annual_revenue',
        y='risk_score',
        color='likelihood_to_hedge',
        size='years_in_business',
        hover_name='company_name',
        hover_data={
            'industry': True,
            'international_operations': True,
            'current_fx_hedging': True,
            'annual_revenue': ':.2f',
            'risk_score': ':.2f'
        },
        color_discrete_map={
            0: 'blue',
            1: 'red'
        },
        title='Client Hedging Potential Insights',
        labels={
            'annual_revenue': 'Annual Revenue ($)',
            'risk_score': 'Combined Risk Score',
            'likelihood_to_hedge': 'Hedging Likelihood',
            'years_in_business': 'Business Maturity'
        }
    )

    # Customize layout for better readability
    fig.update_layout(
        height=800,
        width=1200,
        template='plotly_white',
        coloraxis_colorbar=dict(
            title='Hedging Likelihood'
        )
    )

    # Add statistical annotations
    total_clients = len(insights_df)
    hedge_clients = len(insights_df[insights_df['likelihood_to_hedge'] == 1])
    hedge_percentage = (hedge_clients / total_clients) * 100

    fig.add_annotation(
        xref='paper',
        yref='paper',
        x=0.05,
        y=0.95,
        text=f"Total Clients: {total_clients}<br>" +
             f"Potential Hedge Clients: {hedge_clients}<br>" +
             f"Hedge Potential: {hedge_percentage:.2f}%",
        showarrow=False,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='grey',
        borderwidth=1
    )

    # Create additional interactive elements
    # Industry breakdown
    industry_breakdown = insights_df.groupby(['industry', 'likelihood_to_hedge']).size().unstack(fill_value=0)
    industry_breakdown['total'] = industry_breakdown.sum(axis=1)
    industry_breakdown['hedge_percentage'] = industry_breakdown[1] / industry_breakdown['total'] * 100

    # Bar chart of industry hedging potential
    industry_fig = px.bar(
        industry_breakdown.reset_index(),
        x='industry',
        y='hedge_percentage',
        title='Hedging Potential by Industry',
        labels={'hedge_percentage': 'Hedging Potential (%)'}
    )

    # Save visualizations
    fig.write_html("client_hedging_insights_scatter.html")
    industry_fig.write_html("industry_hedging_potential.html")

    # Generate comprehensive insights report
    insights_report = {
        'total_clients': total_clients,
        'potential_hedge_clients': hedge_clients,
        'hedge_percentage': hedge_percentage,
        'avg_revenue': insights_df['annual_revenue'].mean(),
        'median_revenue': insights_df['annual_revenue'].median(),
        'industry_breakdown': industry_breakdown['hedge_percentage'].to_dict(),
        'risk_score_stats': {
            'mean': insights_df['risk_score'].mean(),
            'median': insights_df['risk_score'].median(),
            'std': insights_df['risk_score'].std()
        }
    }

    # Save insights to JSON
    import json
    with open('comprehensive_insights.json', 'w') as f:
        json.dump(insights_report, f, indent=4)

    return fig, industry_fig, insights_report

# Generate and display insights
try:
    scatter_fig, industry_fig, insights_report = create_comprehensive_insights_visualization(df)

    # Print key insights
    print("Comprehensive Insights Report:")
    for key, value in insights_report.items():
        print(f"{key}: {value}")

except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()


Comprehensive Insights Report:
total_clients: 1000
potential_hedge_clients: 618
hedge_percentage: 61.8
avg_revenue: 25513109.039759547
median_revenue: 25869413.754909635
industry_breakdown: {'Energy': 59.354838709677416, 'Finance': 59.09090909090909, 'Healthcare': 64.9425287356322, 'Logistics': 66.27906976744185, 'Manufacturing': 62.430939226519335, 'Technology': 57.92682926829268}
risk_score_stats: {'mean': 39.12661703525662, 'median': 38.84650277061033, 'std': 15.807009705442846}


In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

def create_basic_scatter_plot(df):
    # Ensure numeric columns
    df['annual_revenue'] = pd.to_numeric(df['annual_revenue'], errors='coerce')
    df['currency_exposure_risk'] = pd.to_numeric(df['currency_exposure_risk'], errors='coerce')

    # Drop any rows with NaN values
    df_clean = df.dropna(subset=['annual_revenue', 'currency_exposure_risk'])

    # Create basic scatter plot
    fig = px.scatter(
        df_clean,
        x='annual_revenue',
        y='currency_exposure_risk',
        color='likelihood_to_hedge',
        title='Client Hedging Potential Scatter Plot'
    )

    # Save the plot
    fig.write_html("basic_scatter_plot.html")

    # Print basic information
    print("Scatter Plot Generated Successfully!")
    print(f"Total data points: {len(df_clean)}")
    print(f"Columns used: annual_revenue, currency_exposure_risk, likelihood_to_hedge")

    return fig

# Debugging function
def debug_dataframe(df):
    print("\nDataFrame Columns:")
    print(df.columns)

    print("\nColumn Data Types:")
    print(df.dtypes)

    print("\nFirst few rows:")
    print(df.head())

    print("\nBasic Statistics:")
    print(df.describe())

# Main execution
try:
    # Debug the dataframe first
    debug_dataframe(df)

    # Create the scatter plot
    scatter_plot = create_basic_scatter_plot(df)

except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()


DataFrame Columns:
Index(['client_id', 'company_name', 'industry', 'annual_revenue',
       'years_in_business', 'international_operations', 'current_fx_hedging',
       'market_volatility_index', 'currency_exposure_risk',
       'likelihood_to_hedge', 'revenue_to_years_ratio', 'risk_score'],
      dtype='object')

Column Data Types:
client_id                     int64
company_name                 object
industry                     object
annual_revenue              float64
years_in_business             int64
international_operations      int64
current_fx_hedging            int64
market_volatility_index     float64
currency_exposure_risk      float64
likelihood_to_hedge           int64
revenue_to_years_ratio      float64
risk_score                  float64
dtype: object

First few rows:
   client_id                  company_name    industry  annual_revenue  \
0          1                   Smith Group  Healthcare    5.987341e+06   
1          2            Alexander and Sons   Logisti

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

def create_interactive_dashboard(df):
    # Create subplot layout
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Hedging Likelihood by Revenue',
            'Currency Exposure Risk Distribution',
            'Industry Hedging Potential',
            'Years in Business vs Hedging'
        )
    )

    # 1. Scatter Plot: Revenue vs Hedging Likelihood
    scatter = go.Scatter(
        x=df['annual_revenue'],
        y=df['currency_exposure_risk'],
        mode='markers',
        marker=dict(
            color=df['likelihood_to_hedge'],
            colorscale='Viridis',
            showscale=True
        ),
        text=[f"Company: {company}<br>Industry: {industry}"
              for company, industry in zip(df['company_name'], df['industry'])],
        hoverinfo='text+x+y'
    )
    fig.add_trace(scatter, row=1, col=1)

    # 2. Histogram: Currency Exposure Risk Distribution
    risk_hist = go.Histogram(
        x=df['currency_exposure_risk'],
        nbinsx=20,
        marker_color='blue',
        opacity=0.7
    )
    fig.add_trace(risk_hist, row=1, col=2)

    # 3. Bar Chart: Industry Hedging Potential
    industry_hedging = df.groupby('industry')['likelihood_to_hedge'].mean().sort_values(ascending=False)
    industry_bar = go.Bar(
        x=industry_hedging.index,
        y=industry_hedging.values,
        marker_color='green'
    )
    fig.add_trace(industry_bar, row=2, col=1)

    # 4. Box Plot: Years in Business vs Hedging
    box_plot = go.Box(
        x=df['likelihood_to_hedge'],
        y=df['years_in_business'],
        marker_color='red'
    )
    fig.add_trace(box_plot, row=2, col=2)

    # Update layout for better visualization
    fig.update_layout(
        height=800,
        width=1200,
        title_text="Comprehensive Hedging Potential Dashboard",
        showlegend=False
    )

    # Axis labels
    fig.update_xaxes(title_text="Annual Revenue", row=1, col=1)
    fig.update_yaxes(title_text="Currency Exposure Risk", row=1, col=1)

    fig.update_xaxes(title_text="Currency Exposure Risk", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)

    fig.update_xaxes(title_text="Industry", row=2, col=1)
    fig.update_yaxes(title_text="Hedging Potential", row=2, col=1)

    fig.update_xaxes(title_text="Hedging Likelihood", row=2, col=2)
    fig.update_yaxes(title_text="Years in Business", row=2, col=2)

    # Generate insights
    insights = {
        'total_clients': len(df),
        'potential_hedge_clients': len(df[df['likelihood_to_hedge'] == 1]),
        'hedge_percentage': len(df[df['likelihood_to_hedge'] == 1]) / len(df) * 100,
        'avg_revenue': df['annual_revenue'].mean(),
        'avg_currency_risk': df['currency_exposure_risk'].mean(),
        'top_hedging_industries': industry_hedging.head().to_dict()
    }

    # Save dashboard and insights
    fig.write_html("hedging_potential_dashboard.html")

    # Save insights to JSON
    import json
    with open('hedging_insights.json', 'w') as f:
        json.dump(insights, f, indent=4)

    return fig, insights

# Execute dashboard creation
try:
    dashboard, insights = create_interactive_dashboard(df)

    # Print insights
    print("Hedging Potential Insights:")
    for key, value in insights.items():
        print(f"{key}: {value}")

except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

Hedging Potential Insights:
total_clients: 1000
potential_hedge_clients: 618
hedge_percentage: 61.8
avg_revenue: 25513109.039759547
avg_currency_risk: 48.65815629408703
top_hedging_industries: {'Logistics': 0.6627906976744186, 'Healthcare': 0.6494252873563219, 'Manufacturing': 0.6243093922651933, 'Energy': 0.5935483870967742, 'Finance': 0.5909090909090909}
